# **Procesamiento del habla**

## Estructura de sub-etapas

---

```
├── Reconocimiento del lenguaje
│   ├── Captura de texto
│   └── Captura de audio
├── Procesamiento del lenguaje
│   ├── Preprocesamiento de texto
│   └── Analisis de texto
├── Gestion de respuesta
│   ├── Gestion del estado
│   ├── Motor de respuesta
│   └── Personalizacion
├── Texto a voz
│   └── Conversion texto a voz

```


#### _Estructura operativa para cada sub-etapa:_

**Selección de técnicas de modelado:** En este caso se utilizara [] para la etapa de reconocimiento de voz, [] para el procesamiento y analisis de texto y [] para la conversion de texto a voz.

**Generación de un diseño de comprobación:** Para elegir el modelo correcto, este devera ser el que pondere mas alto en un promedio de las diferentes metricas de evaluacion del modelo.

**Generación de los modelos:** Se definiran y configuraran los parametros del modelo para pasar a la etapa de ejecucion y descripcion.

**Evaluación del modelo:** Se evaluaran los diferentes modelos de manera individual en busqueda de optimizar sus parametros y escoger la combinacion mas optima entre modelos / parametros individuales.

### Librerias y paquetes

#### *Entorno virtuales*

In [23]:
# Create and activate virtual env for speech processing stage
!python3 -m venv proc_habla
!source proc_habla/bin/activate

#### *Instalacion*

In [24]:
# Librerias de preprocesamiento
!pip3 install sounddevice
!pip3 install scipy
!pip3 install pydub

# Librerias de reconocimiento de voz
!pip3 install pocketsphinx
!pip3 install SpeechRecognition

# Librerias de generacion de voz
!pip3 install gtts
# Librerias de analisis linguistico
!pip3 install spacy

In [25]:
!python3 -m spacy download es_core_news_sm

  Using cached https://github.com/explosion/spacy-models/releases/download/es_core_news_sm-3.7.0/es_core_news_sm-3.7.0-py3-none-any.whl (12.9 MB)
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')


#### *Importacion*

In [26]:
# Librerías para reconocimiento de voz y procesamiento de audio
import speech_recognition as sr
import sounddevice as sd
from scipy.io.wavfile import write
import queue
import threading
from pydub import AudioSegment

# Librerías para manipulación de archivos y compresión
import wave
import zipfile
import os
import subprocess

# Librerías para procesamiento de datos y análisis
import numpy as np
import pandas as pd
import spacy
import re

# Librería para generación de voz
from gtts import gTTS

# Importación de funciones comunes a otros cuadernos
from funciones_comunes import common_functions

## 1. Reconocimiento del Habla

___

In [27]:
# Ruta del archivo zip descargado
ruta_modelo_comprimido_pocketsphinx_esp = "../datos/brutos/modelos_proc_habla/modelo_esp.zip"

# Directorio donde se colocarán los archivos descomprimidos
ruta_modelo_descomprimido_pocketsphinx_esp = "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/speech_recognition/pocketsphinx-data/es-ES/"

# Descomprimir el archivo zip
with zipfile.ZipFile(ruta_modelo_comprimido_pocketsphinx_esp, 'r') as zip_ref:
    zip_ref.extractall(ruta_modelo_descomprimido_pocketsphinx_esp)

print("Archivos descomprimidos en:", ruta_modelo_descomprimido_pocketsphinx_esp)

Archivos descomprimidos en: /Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/speech_recognition/pocketsphinx-data/es-ES/


In [28]:
try:
    subprocess.run(["sudo", "apt-get", "install", "ffmpeg"])
    print("FFmpeg instalado exitosamente en sistemas basados en Debian.")
except Exception as e:
    print("Error al instalar FFmpeg:", e)
try:
    subprocess.run(["brew", "install", "ffmpeg"])
    print("FFmpeg instalado exitosamente en macOS utilizando Homebrew.")
except Exception as e:
    print("Error al instalar FFmpeg:", e)

FFmpeg instalado exitosamente en sistemas basados en Debian.


sudo: a terminal is required to read the password; either use the -S option to read from standard input or configure an askpass helper
sudo: a password is required


FFmpeg instalado exitosamente en macOS utilizando Homebrew.


To reinstall 7.0.1, run:
  brew reinstall ffmpeg


### DataFrames

In [29]:
consulta_dict = {}

In [30]:
path_csv_rutas_verduras = '../datos/procesaodos/VerdurasporSupermercado.csv'
df_frutas_verduras = pd.read_csv(path_csv_rutas_verduras)

### a. Captura de entradas de texto

In [31]:
#Conexion con api de fast-api
# aui iria la conexion con un script encargado de conectar con la api de fast-api

In [32]:
texto_prueba = 'estoy buscando tomates en oferta pero que esten bien rojitos :p'

### b. Captura de Audio

In [33]:
SAMPLE_RATE = 44100  # Tasa de muestreo
DURATION = 10  # Duración de la grabación en segundos
AUDIO_FILES_PATH = "../datos/brutos/audios_proc_habla/"
FILENAME = "output"  # Nombre del archivo de salida
EXTENCION_ENTRADA = '.wav'
EXTENCION_SALIDA = '.aiff'

In [34]:
ruta_audio_entrada = AUDIO_FILES_PATH + FILENAME + EXTENCION_ENTRADA
ruta_audio_entrada_convertido = AUDIO_FILES_PATH + 'waw_conv_' + FILENAME + EXTENCION_SALIDA

In [35]:
def record_audio(audio_file_path = AUDIO_FILES_PATH, filename = FILENAME, duration = DURATION, sample_rate = SAMPLE_RATE):
    print("Grabando...")
    # Grabar audio con 1 canal (mono)
    recording = sd.rec(int(duration * sample_rate), samplerate=sample_rate, channels=1)
    sd.wait()  # Esperar a que termine la grabación
    print("Grabación finalizada")
    write(ruta_audio_entrada, sample_rate, recording)
    return filename

In [36]:
def convert_to_wav(input_file = ruta_audio_entrada, output_file = ruta_audio_entrada_convertido):
    sound = AudioSegment.from_file(input_file)
    sound.export(output_file, format="aiff")

In [37]:
def chatbot():
    record_audio()

In [38]:
chatbot()

Grabando...
Grabación finalizada


In [39]:
# Uso de la función convert_to_wav
convert_to_wav(ruta_audio_entrada, ruta_audio_entrada_convertido)

#### Reconocimiento del Habla (ASR - Automatic Speech Recognition):

In [40]:
# Crear un objeto Recognizer
recognizer = sr.Recognizer()

In [41]:
def recognize_speech():
    recognizer = sr.Recognizer()

    try:
        with sr.AudioFile(ruta_audio_entrada_convertido) as source:
            audio_data = recognizer.record(source)
            text = recognizer.recognize_sphinx(audio_data, language="es-ES")
            print("Texto reconocido:", text)
    except sr.UnknownValueError:
        print("No se pudo entender el audio")
    except sr.RequestError as e:
        print("Error al solicitar resultados de reconocimiento de voz; {0}".format(e))

In [42]:
# Ejecutar la función para reconocer el discurso
recognize_speech()

Error al solicitar resultados de reconocimiento de voz; missing PocketSphinx language model parameters directory: "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/speech_recognition/pocketsphinx-data/es-ES/acoustic-model"


In [43]:
# Abrir el archivo de audio
def transcribe_audio(audio_lang = 'es-ES'):
    with sr.AudioFile(ruta_audio_entrada_convertido) as source:
        # Escuchar el audio (en inglés)
        audio = recognizer.record(source)

        # Utilizar Google Speech Recognition para transcribir el audio
        try:
            text = recognizer.recognize_google(audio, audio_lang)
            print("Transcripción: ", text)
            return text
        except sr.UnknownValueError:
            print("No se pudo entender el audio")
        except sr.RequestError as e:
            print("Error al solicitar resultados del servicio Google Speech Recognition; {0}".format(e))

In [44]:
#transcribe_audio()

In [45]:
# Transcribir y mostrar el resultado
#text = transcribe_audio(audio_file_path, audio_lang)

## 2. Procesamiento del Lenguaje Natural

___

In [46]:
# Cargar el modelo de spaCy para español
nlp = spacy.load("es_core_news_sm")

### Preprocesamiento de texto


In [47]:
# Lista de abreviaturas y formas no abreviadas de unidades de medida de peso
'''

# unidades_medida_peso = ['g', 'gr', 'kg', 'mg', 'µg', 't', 'lb', 'oz', 'cwt',
                        #'gramos', 'kilogramos', 'miligramos', 'microgramos',
                        #'toneladas', 'libras', 'onzas', 'quintales']

'''

"\n\n# unidades_medida_peso = ['g', 'gr', 'kg', 'mg', 'µg', 't', 'lb', 'oz', 'cwt',\n                        #'gramos', 'kilogramos', 'miligramos', 'microgramos',\n                        #'toneladas', 'libras', 'onzas', 'quintales']\n\n"

In [48]:
def procesar_texto(texto):
    texto = texto.lower()
    texto = re.sub(r'[,\.;:!\-*#@$!+_%^&`~]', '', texto)
    texto = re.sub(r'\s+', ' ', texto)

    thresgold = 1

    doc = nlp(texto)

    tokens = [token.text for token in doc]
    stopwords = spacy.lang.es.stop_words.STOP_WORDS
    filtered_tokens = [
        token.text for token in doc
        if token.text not in stopwords
        and token.pos_ in ('NOUN', 'ADJ', 'ADP')
        and len(token.text) > thresgold
        ]#and token.text not in unidades_medida_peso
    lemmas = [token.lemma_ for token in doc if token.text in filtered_tokens]
    pos_tags = [(token.text, token.pos_) for token in doc if token.text in filtered_tokens]

    return {
        "tokens": tokens,
        "lemmas": lemmas,
        "filtered_tokens": filtered_tokens,
        "pos_tags": pos_tags,
    }


**Normalización del texto (eliminación de ruido, corrección ortográfica, etc.)**

In [49]:
texto_analizado = procesar_texto(texto_prueba)

**Tokenización**

In [50]:
tokens = texto_analizado['tokens']

In [51]:
tokens

['estoy',
 'buscando',
 'tomates',
 'en',
 'oferta',
 'pero',
 'que',
 'esten',
 'bien',
 'rojitos',
 'p']

**Eliminación de stop words.**


In [52]:
tokens_filtrados = texto_analizado['filtered_tokens']

In [53]:
tokens_filtrados

['tomates', 'oferta', 'rojitos']

**Lematización y stemming**

In [54]:
lemmas_filtrados = texto_analizado['lemmas']

In [55]:
lemmas_filtrados

['tomate', 'oferta', 'rojito']

Etiquetado de estructuras gramaticales:

In [56]:
pos_tags_filtrados = texto_analizado['pos_tags']

In [57]:
pos_tags_filtrados

[('tomates', 'NOUN'), ('oferta', 'NOUN'), ('rojitos', 'ADJ')]

### Análisis de texto


Detectar presencia de palabras de productos lematizadas dentro de dataframe

In [58]:
df_frutas_verduras['producto_tokens_lemmas'] = df_frutas_verduras['Producto'].apply(lambda x: procesar_texto(x)['lemmas'])

In [59]:
dict_count_coincidences = {}

In [60]:
for index, row in df_frutas_verduras.iterrows():
    for token_producto in row['producto_tokens_lemmas']:
        if token_producto in lemmas_filtrados:
            if token_producto not in dict_count_coincidences:
                dict_count_coincidences[index] = 1
            else:
                dict_count_coincidences[index] += 1

In [61]:
series_ordered_count_coincidences = pd.Series(dict_count_coincidences)

Análisis de Sentimiento: Determinar la emoción o tono del texto.


In [62]:
#para futuras versiones

Detección de Intenciones (Intent Detection): Identificar la intención del usuario utilizando modelos como BERT, GPT, RASA, etc.


In [63]:
#para futuras versiones

## 3. Gestión del Diálogo

___

### Módulo de Gestión de Estado:
Llevar un registro del contexto y estado del diálogo para mantener conversaciones coherentes.


In [75]:
consulta_dict['id_consulta'] = None
consulta_dict['id_cliente'] = None
consulta_dict['formato_consulta'] = None
consulta_dict['transcripción_audio'] = None
consulta_dict['entrada_texto'] = texto_prueba
consulta_dict['entrada_lemas_filtrados'] = lemmas_filtrados
consulta_dict['dict_producto_indice_considencias'] = dict_count_coincidences
consulta_dict['card_recomendacion'] = None
audio_recomendacion = None

### Motor de Respuesta:


Generación de Respuestas: Utilizar modelos generativos (como GPT-3) o respuestas predefinidas según las intenciones y entidades detectadas.


In [65]:
df_frutas_verduras['Precio'] = df_frutas_verduras['Precio'].apply(common_functions.limpiar_signo_peso)

In [74]:
# Asegúrate de que series_ordered_count_coincidences es una Serie ordenada
series_ordered_count_coincidences = series_ordered_count_coincidences.sort_values(ascending=False)

# Inicialización de la recomendación
recomendacion = 'Los productos recomendados en base a su consulta y caracteristicas mensionadas son:\n'

# Lista para almacenar las recomendaciones
lista_string_reco_supers_prods = []

# Iterar sobre los índices de las coincidencias ordenadas
for count, x in enumerate(series_ordered_count_coincidences.index, start=1):
    # Extraer los detalles del producto usando el índice
    producto, supermercado, precio = df_frutas_verduras.loc[x, ['Producto', 'Supermercado', 'Precio']]
    
    # Añadir la recomendación a la lista
    lista_string_reco_supers_prods.append(f'En el lugar {count}: {producto} en {supermercado} a {precio} pesos\n')

# Concatenar todas las recomendaciones en un solo string
recomendacion += ''.join(lista_string_reco_supers_prods)

recomendacion = common_functions.redondear_numeros(recomendacion)

consulta_dict['card_recomendacion'] = recomendacion

print(recomendacion)


Los productos recomendados en base a su consulta y caracteristicas mensionadas son:
En el lugar 1: Tomates secos 70 g. en Carrefour a 2229 pesos
En el lugar 2: Tomate seco 100 g. en Carrefour a 1799 pesos
En el lugar 3: Tomates secos 70 g. en Carrefour a 2229 pesos
En el lugar 4: Tomates Secos El Peoncito x 100 g. en La Anonima a 799 pesos
En el lugar 5: Tomates Peritas Deshidratados Nature Food x 150 g. en La Anonima a 4750 pesos
En el lugar 6: Tomates Deshidratados El Peoncito x 100 g. en La Anonima a 2150 pesos
En el lugar 7: Tomates Secos El Peoncito x 100 g. en La Anonima a 4000 pesos



Selección de Respuestas: Elegir la mejor respuesta entre varias opciones generadas.


In [67]:
#Para proximas versiones

### Personalización y Contexto: Adaptar las respuestas en función del historial del usuario y el contexto actual.

In [68]:
#Para proximas versiones

## 4. Texto a voz

___

### Conversión de Texto a Voz (TTS): Utilizar servicios como Google Text-to-Speech, Amazon Polly, o frameworks como Tacotron para convertir el texto generado en voz.


In [69]:
tts = gTTS(text=recomendacion, lang='es')

In [ ]:
# Guardar el audio en un objeto BytesIO
audio_buffer = io.BytesIO()
tts.write_to_fp(audio_buffer)

# Asegurarse de que el puntero esté al principio del buffer
audio_buffer.seek(0)

# Crear un diccionario y guardar el audio en él
consulta_dict = {'recomendacion_audio': audio_buffer}

In [70]:
tts.save("../datos/procesaodos/proc_habla/output.mp3")